# Data Collection - Conceptual Captions
In this document, I'll download the images from Google's Conceptual Captions dataset. (https://ai.google.com/research/ConceptualCaptions/download)

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../DATA/Train_GCC-training.tsv', delimiter='\t', header=None, names = ['description', 'url'])

In [3]:
val_df = pd.read_csv('../DATA/Validation_GCC-1.1.0-Validation.tsv', delimiter = '\t', header = None, names = ['description', 'url'])

### Download images
Now I'll make a call to individual url to download images to the local drive.

In [4]:
import requests
import sys
import time
from PIL import Image
import os

In [5]:
path = '../DATA/IMAGES/Conceptual_Captions'

In [6]:
flist = os.listdir(path)

In [12]:
# downloading images
def download_img(ind, url, path, flist = None, pause = False): 
    if flist == None: 
        flist = os.listdir(path)
    
    print(f"Processing {ind} ", end = '...')
    
    if f"{ind}.jpg" in flist: 
        print(f"{ind}.jpg exists.")
        status = 'saved'
    
    if url.startswith('http'):
        
        try: 
            # save the file
            fpath = f"{path}/{ind}.jpg"

            resp = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'})
                        
            with open(fpath, 'wb') as fp:
                fp.write(resp.content)

            status = 'saved'

            try:
                # resizing image
                img = Image.open(fpath)
                img.thumbnail((500, 500))
                img.save(fpath)
                status = 'complete'
                print('Complete')

            except: 
                print(f'Failed to resize. \n{sys.exc_info()[0]}')
                status = 'failed: resizing'

        except:
            print(f'Failed to save. \n{sys.exc_info()[0]}')
            status = 'failed: saving'


    else:
        status = f'failed: invalid_url {url[:15]}...'
    
    if pause: 
        pause = np.random.randint(1, 60)
        time.sleep(pause) # buffer
        
    return status

In [13]:
tmp = train_df.copy()

In [14]:
tmp['status'] = None

In [ ]:
tmp['url'][83]

In [ ]:
for ind, url in enumerate(tmp['url']): 
    sta080tus = download_img(ind, url, path, flist = flist) 
    if status: 
        tmp['status'][ind] = status

0.jpg exists.
1.jpg exists.
2.jpg exists.
3.jpg exists.
4.jpg exists.
5.jpg exists.
6.jpg exists.
7.jpg exists.
8.jpg exists.
9.jpg exists.
10.jpg exists.
Processing 11 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
11 failed: resizing
Processing 12 ...Failed to save. 
<class 'requests.exceptions.ConnectionError'>
12 failed: saving
13.jpg exists.
14.jpg exists.
15.jpg exists.
16.jpg exists.
Processing 17 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
17 failed: resizing
Processing 18 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
18 failed: resizing
19.jpg exists.
20.jpg exists.
21.jpg exists.
22.jpg exists.
23.jpg exists.
Processing 24 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
24 failed: resizing
25.jpg exists.
26.jpg exists.
27.jpg exists.
28.jpg exists.
29.jpg exists.
30.jpg exists.
31.jpg exists.
Processing 32 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
32 failed: resizing
33.jpg exists.
34.jpg exists.
35.jpg exist

D:\ProgramData\Anaconda3\envs\workspace-gpu\lib\site-packages\PIL\Image.py:2929: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
148 failed: resizing
Processing 149 ...Failed to save. 
<class 'requests.exceptions.SSLError'>
149 failed: saving
Processing 150 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
150 failed: resizing
Processing 151 ...Complete
151 complete
Processing 152 ...Complete
152 complete
Processing 153 ...Complete
153 complete
Processing 154 ...Complete
154 complete
Processing 155 ...Complete
155 complete
Processing 156 ...Complete
156 complete
Processing 157 ...Complete
157 complete
Processing 158 ...Complete
158 complete
Processing 159 ...Complete
159 complete
Processing 160 ...Complete
160 complete
Processing 161 ...Complete
161 complete
Processing 162 ...Complete
162 complete
Processing 163 ...Complete
163 complete
Processing 164 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
164 failed: resizing
Processing 165 ...Failed to save. 
<class 'requests.exceptions.ConnectionError'>
165 failed: saving
Processing 166 ...Co

Processing 327 ...Complete
327 complete
Processing 328 ...Complete
328 complete
Processing 329 ...Complete
329 complete
Processing 330 ...Complete
330 complete
Processing 331 ...Complete
331 complete
Processing 332 ...Complete
332 complete
Processing 333 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
333 failed: resizing
Processing 334 ...Complete
334 complete
Processing 335 ...Complete
335 complete
Processing 336 ...Complete
336 complete
Processing 337 ...Complete
337 complete
Processing 338 ...Complete
338 complete
Processing 339 ...Complete
339 complete
Processing 340 ...Complete
340 complete
Processing 341 ...Complete
341 complete
Processing 342 ...Complete
342 complete
Processing 343 ...Complete
343 complete
Processing 344 ...Complete
344 complete
Processing 345 ...Complete
345 complete
Processing 346 ...Complete
346 complete
Processing 347 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
347 failed: resizing
Processing 348 ...Complete
348 complete
Processing

Processing 509 ...Complete
509 complete
Processing 510 ...Complete
510 complete
Processing 511 ...Complete
511 complete
Processing 512 ...Complete
512 complete
Processing 513 ...Complete
513 complete
Processing 514 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
514 failed: resizing
Processing 515 ...Complete
515 complete
Processing 516 ...Complete
516 complete
Processing 517 ...Complete
517 complete
Processing 518 ...Complete
518 complete
Processing 519 ...Complete
519 complete
Processing 520 ...Complete
520 complete
Processing 521 ...Complete
521 complete
Processing 522 ...Complete
522 complete
Processing 523 ...Complete
523 complete
Processing 524 ...Complete
524 complete
Processing 525 ...Complete
525 complete
Processing 526 ...Complete
526 complete
Processing 527 ...Complete
527 complete
Processing 528 ...Complete
528 complete
Processing 529 ...Complete
529 complete
Processing 530 ...Complete
530 complete
Processing 531 ...Complete
531 complete
Processing 532 ...Failed 

...

Complete
25971 complete
Processing 25972 ...Complete
25972 complete
Processing 25973 ...Complete
25973 complete
Processing 25974 ...Complete
25974 complete
Processing 25975 ...Complete
25975 complete
Processing 25976 ...Complete
25976 complete
Processing 25977 ...Complete
25977 complete
Processing 25978 ...Complete
25978 complete
Processing 25979 ...Complete
25979 complete
Processing 25980 ...Complete
25980 complete
Processing 25981 ...Complete
25981 complete
Processing 25982 ...Complete
25982 complete
Processing 25983 ...Complete
25983 complete
Processing 25984 ...Complete
25984 complete
Processing 25985 ...Complete
25985 complete
Processing 25986 ...Complete
25986 complete
Processing 25987 ...Complete
25987 complete
Processing 25988 ...Complete
25988 complete
Processing 25989 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
25989 failed: resizing
Processing 25990 ...Complete
25990 complete
Processing 25991 ...Complete
25991 complete
Processing 25992 ...Complete
25992 comple

Processing 26139 ...Complete
26139 complete
Processing 26140 ...Complete
26140 complete
Processing 26141 ...Complete
26141 complete
Processing 26142 ...Complete
26142 complete
Processing 26143 ...Complete
26143 complete
Processing 26144 ...Complete
26144 complete
Processing 26145 ...Complete
26145 complete
Processing 26146 ...Complete
26146 complete
Processing 26147 ...Complete
26147 complete
Processing 26148 ...Complete
26148 complete
Processing 26149 ...Complete
26149 complete
Processing 26150 ...Complete
26150 complete
Processing 26151 ...Complete
26151 complete
Processing 26152 ...Complete
26152 complete
Processing 26153 ...Complete
26153 complete
Processing 26154 ...Complete
26154 complete
Processing 26155 ...Complete
26155 complete
Processing 26156 ...Complete
26156 complete
Processing 26157 ...Complete
26157 complete
Processing 26158 ...Complete
26158 complete
Processing 26159 ...Complete
26159 complete
Processing 26160 ...Complete
26160 complete
Processing 26161 ...Complete
261

Processing 26303 ...Complete
26303 complete
Processing 26304 ...Complete
26304 complete
Processing 26305 ...Complete
26305 complete
Processing 26306 ...Complete
26306 complete
Processing 26307 ...Complete
26307 complete
Processing 26308 ...Complete
26308 complete
Processing 26309 ...Complete
26309 complete
Processing 26310 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
26310 failed: resizing
Processing 26311 ...Complete
26311 complete
Processing 26312 ...Complete
26312 complete
Processing 26313 ...Complete
26313 complete
Processing 26314 ...Complete
26314 complete
Processing 26315 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
26315 failed: resizing
Processing 26316 ...Complete
26316 complete
Processing 26317 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
26317 failed: resizing
Processing 26318 ...Complete
26318 complete
Processing 26319 ...Complete
26319 complete
Processing 26320 ...Complete
26320 complete
Processing 26321 ...Complete
26321 complete

Processing 26473 ...Complete
26473 complete
Processing 26474 ...Complete
26474 complete
Processing 26475 ...Complete
26475 complete
Processing 26476 ...Complete
26476 complete
Processing 26477 ...Complete
26477 complete
Processing 26478 ...Complete
26478 complete
Processing 26479 ...Complete
26479 complete
Processing 26480 ...Complete
26480 complete
Processing 26481 ...Complete
26481 complete
Processing 26482 ...Complete
26482 complete
Processing 26483 ...Complete
26483 complete
Processing 26484 ...Complete
26484 complete
Processing 26485 ...Complete
26485 complete
Processing 26486 ...Complete
26486 complete
Processing 26487 ...Complete
26487 complete
Processing 26488 ...Complete
26488 complete
Processing 26489 ...Complete
26489 complete
Processing 26490 ...Complete
26490 complete
Processing 26491 ...Complete
26491 complete
Processing 26492 ...Complete
26492 complete
Processing 26493 ...Complete
26493 complete
Processing 26494 ...Complete
26494 complete
Processing 26495 ...Complete
264

Processing 26636 ...Complete
26636 complete
Processing 26637 ...Complete
26637 complete
Processing 26638 ...Complete
26638 complete
Processing 26639 ...Complete
26639 complete
Processing 26640 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
26640 failed: resizing
Processing 26641 ...Failed to resize. 
<class 'PIL.UnidentifiedImageError'>
26641 failed: resizing
Processing 26642 ...Complete
26642 complete
Processing 26643 ...Complete
26643 complete
Processing 26644 ...Complete
26644 complete
Processing 26645 ...Complete
26645 complete
Processing 26646 ...Complete
26646 complete
Processing 26647 ...Complete
26647 complete
Processing 26648 ...Complete
26648 complete
Processing 26649 ...Complete
26649 complete
Processing 26650 ...Failed to save. 
<class 'KeyboardInterrupt'>
26650 failed: saving
Processing 26651 ...Complete
26651 complete
Processing 26652 ...Complete
26652 complete
Processing 26653 ...Complete
26653 complete
Processing 26654 ...Complete
26654 complete
Processing 2